Notes on the time cell dataset, written Sept. 20, 2021. -Daniel

__General notes.__
- There are 10 subjects with 12 complete sessions of data, all epilepsy patients implanted with Behnke-Fried electrodes by Itzhak Fried at UCLA. These subjects are: U518 (2 sessions), U521, U526, U527, U530 (2 sessions), U532, U536, U537, U539, and U540. 
  - I collected 1.5 sessions of data on an 11th subject, U515 (coincidentally the first we tested), but do not recommend analyzing this data for reasons described in the testing notes.
- During each ~1hr session, subjects first completed a 2-3 trial tutorial in a tutorial-only environment. Then they did 36 trials in the main maze, which is what I have used for all analyses.
- There are 476 single- and multi-units across these 10 subjects
  - Includes 109 hippocampal units, 204 units in the surrounding MTL (entorhinal cortex, amygdala, or parahippocampal gyrus), 75 lateral temporal lobe units, 69 prefrontal units, and 19 occipital or parietal units from a single subject (U518).
- All subjects after U518 played the same game version, Goldmine 1.0.2. U518 played Goldmine 1.0 during their first session and Goldmine 1.0.1 during their second session.
  - Changes between versions are minimal, with the most notable being that a slightly different environment was used in 1.0.2 than previously, although with the same basic maze shape.
  - All changes from 1.0 to 1.0.1 are noted in the testing notes for U518/ses1, and all changes from 1.0.1 to 1.0.2 are noted in the testing notes for U521/ses0. 

__Raw data organization.__
- Raw data are stored in /data7/goldmine/data, and all sessions follow the same directory structure, as follows.
  - */data7/goldmine/data/subj/sess/events*: testing_notes.txt has all my notes on the session and is probably a good first document to read. events.jsonl file contains all behavioral events that are logged at gametime, along with sync pulse times that are used to align behavioral events to EEG (accurate within ~1ms). Sometimes there are image files in this dir that the UCLA research techs sent me, with electrode mapping info or their internal testing notes.
  - */data7/goldmine/data/subj/sess/macro_lfps*: Macro EEG data, if it exists. There will also be a timestamps file.
  - */data7/goldmine/data/subj/sess/micro_lfps*: Microwire EEG data, which follows the format CSC*.mat, where * = the channel number from 1 to however many micros were implanted. Each depth probe bundles 8 recording wires, so micros 1-8 are in the same region, 9-16 are in the same region, etc. anatleads.txt lists the loations of each micro in the brain, using UCLA's brain region acronyms. The first letter is always L or R for left and right, respectively. e.g. LAH = left anterior hippocampus. lfpTimeStamps_micro.mat contains the timestamps for the microwires, as logged by the EEG recording system.
  - */data7/goldmine/data/subj/sess/spikes*: The wave_clus3 spike sorting algorithm (Chaure, Rey & Quiroga, J Neurophysiol 2018) is run on each microwire channel to yield automatic spike sorting results (/wave_clus3_autosort). Then I went through each channel and accepted, merged, or rejected each unit by hand (wave_clus3_sortbyhand). The latter contain the spiking data that I use in all analyses. I have included both single- and multi-units in wave_clus3_sortbyhand, but have not made a quantitative distinction between these.
  - */data7/goldmine/data/subj/sess/sync*: sync_channel_micro.mat contains analog sync pulses with the same timestamps as the microwires. This file is used alongside synctimes in events.jsonl to align behavioral events to EEG.

__Preprocessed data organization.__
- Preprocessed data is stored in /data7/goldmine/analysis. The following files exist for each data session.
  - */data7/goldmine/analysis*: subj_sess-events_formatted.pkl is created by events_preproc.format_events() and is basically just the events.jsonl dataframe after aligning times to EEG timestamps.  In practice you probably won't ever need to load this file. subj_sess-Events.pkl is loaded by events_proc.load_events() and explained in detail below. subj_sess-EventSpikes.pkl is loaded by time_bin_analysis.load_event_spikes() and also explained in the cells below.
  - */data7/goldmine/spikes*: Contains one file for each spike-sorted single- or multi-unit. File name is like '{}_{}_CSC{}_unit{}-spikes.pkl'.format(subj, sess, chan, unit). Each file contains a pandas Series object with the region location ('hemroi') and a vector ('spike_times') that lists all spike times, in ms, aligned to the event times in analysis/events/subj_sess-Events.pkl dataframes. These files are created by spike_preproc.format_spikes(), and you can load the saved files with spike_preproc.load_spikes().

In [1]:
# General
import sys
import os.path as op
from time import time
from collections import OrderedDict as od
from glob import glob
import itertools
import warnings
from importlib import reload

# Scientific
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

# Stats
import statsmodels.api as sm
from sklearn.utils.fixes import loguniform
import scipy.stats as stats
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import minmax_scale, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix

# Plots
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.patches as patches
# from mpl_toolkits.mplot3d import Axes3D
mpl.rcParams['grid.linewidth'] = 0.1
mpl.rcParams['grid.alpha'] = 0.75
mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['lines.markersize'] = 3
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['xtick.major.width'] = 0.8
mpl.rcParams['ytick.major.width'] = 0.8
colors = ['1f77b4', 'd62728', '2ca02c', 'ff7f0e', '9467bd', 
          '8c564b', 'e377c2', '7f7f7f', 'bcbd22', '17becf']
mpl.rcParams['axes.prop_cycle'] = mpl.cycler('color', colors)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.formatter.offset_threshold'] = 2
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['axes.labelpad'] = 8
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.grid'] = False
mpl.rcParams['axes.axisbelow'] = True
mpl.rcParams['legend.loc'] = 'upper right'
mpl.rcParams['legend.fontsize'] = 14
mpl.rcParams['legend.frameon'] = False
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['figure.titlesize'] = 16
mpl.rcParams['figure.figsize'] = (10, 4) 
mpl.rcParams['figure.subplot.wspace'] = 0.25 
mpl.rcParams['figure.subplot.hspace'] = 0.25 
mpl.rcParams['font.sans-serif'] = ['Helvetica']
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['pdf.fonttype'] = 42

# Personal
sys.path.append('/home1/dscho/code/general')
sys.path.append('/home1/dscho/code/projects/manning_replication')
sys.path.append('/home1/dscho/code/projects')
import data_io as dio
import array_operations as aop
from eeg_plotting import plot_trace, plot_trace2
from time_cells import spike_sorting, spike_preproc, events_preproc, events_proc, time_bin_analysis, pop_decoding

proj_dir = '/data7/goldmine'

In [7]:
# Let's see how many data sessions have been preprocessed.
sessions = np.unique([op.basename(f).split('-')[0] 
                      for f in glob(op.join(proj_dir, 'analysis', 'events', '*-Events.pkl'))])
print('{} subjects, {} sessions'.format(len(np.unique([x.split('_')[0] for x in sessions])), len(sessions)), end='\n'*2)
print('Sessions are: {}'.format(sessions))

10 subjects, 12 sessions

Sessions are: ['U518_ses0' 'U518_ses1' 'U521_ses0' 'U526_ses0' 'U527_ses0' 'U530_ses0'
 'U530_ses1' 'U532_ses0' 'U536_ses0' 'U537_ses0' 'U539_ses0' 'U540_ses0']


In [8]:
# Now we load the events class for an example session, U518_ses0 (i.e. subject U518, first session).
# The same function shows you how this class gets created, if the file is not already saved,
# and there are some preprocessing options you can toggle if desired. But for now we'll just load
# an already-saved file.
subj_sess = 'U518_ses0'

events = events_proc.load_events(subj_sess, proj_dir=proj_dir)

print(events)

Loading saved Events file
------------------------------------
U518_ses0
36 trials completed, 36 good

40 gold spawned
34 gold viewed (85%)
13 gold dug (32%)
38 dig attempts (34% accuracy)

dataframe sizes:
	events: (72204, 8)
	event_times: (216, 10)
	positions: (46861, 15)
	base_events: (9371, 7)
	gold_events: (9516, 8)
	gold_spawned: (40, 8)
	dig_events: (38, 8)
	events_behav: (6366, 20)



In [9]:
# The events class is really just a container for a few different events dataframes,
# all of which are listed in the print output above. 
# I'll quickly list these dataframes here and explain what they are.

# events contains the processed output of the raw events.jsonl file that gets written
# at gametime. By 'processed', I mean I have: (1) aligned event timestamps to EEG and
# converted time to ms; (2) removed tutorial events; (3) removed any incomplete trials
# as well as trials where the subject manually paused the game for some reason (in practice
# I excluded ~2% of trials); (4) added columns that track the trial and gameState for each
# recorded event. Otherwise, the rows of this dataframe correspond to events as they are
# recorded by the game; 'time' tells you the time of each event; 'key' tells you what kind
# of event is logged; and 'value' contains a dict with all of the event details.
print('-------------')
print('events.events')
display(events.events.head())

# event_times gives the start, stop timestamps for each gameState,
# within each trial (see the 'time' column).
print('\n------------------')
print('events.event_times')
display(events.event_times.head())

# positions lists the x, y position and head rotation of the subject at
# each moment during the navigation phases of the task.
print('\n----------------')
print('events.positions')
display(events.positions.head())

# base_events corresponds to looping baseInView() calls that are made
# during the Encoding and Retrieval periods of gameplay. Each time
# this function is called, we get information about whether or not the base
# could be seen by the subject from their current position. Visibility is
# blocked by walls and only applies to views of the outside of the base; i.e.,
# when the subject was actually in the base, baseInView is False.
print('\n------------------')
print('events.base_events')
display(events.base_events.head())

# gold_events corresponds to looping goldInView() calls that are made
# during the Encoding and Retrieval periods of gameplay. Each time
# this function is called, we get information about whether or not a particular
# gold was visible to the subject from their current position. Visibility is 
# blocked by walls. During Retrieval, gold object meshes were turned off and so 
# became invisible to the subject, but not to goldInView(), so we get a log of
# when subjects would have seen each gold were it rendered visible.
print('\n----------------')
print('events.gold_events')
display(events.gold_events.head())

# gold_spawned lists each gold item that was spawned at the beginning
# of Encoding, where it was in the maze, and the unique ID used to track it
# in the events, gold_events, and dig_events dataframes.
print('\n----------------')
print('events.gold_spawned')
display(events.gold_spawned.head())

# dig_events lists every time the subject chose to dig during Retrieval,
# which gold they were closest to at the time of the dig, whether the dig
# was successful (i.e. dig was close to retrieve the gold for points) or not,
# and how far away the gold was in virtual meters.
print('\n----------------')
print('events.dig_events')
display(events.dig_events.head())

# events_behav contains a selection of event variables in 500ms windows,
# and is used for setting up the data matrices for time cell regression
# and population classifier models.
print('\n----------------')
print('events.events_behav')
display(events.events_behav.head())

-------------
events.events


,time,key,value,scene,trial,time_penalty,gameState,bad_trials
0,782611,gameState,{'stateName': 'InitTrial'},SimpleMaze,1,0,InitTrial,
1,782611,playerRespawnTransform,"{'positionX': 16.5, 'positionZ': 1.6, 'rotatio...",SimpleMaze,1,0,InitTrial,
2,782611,baseDoorsOpen,"{'rightDoor': False, 'leftDoor': False, 'middl...",SimpleMaze,1,0,InitTrial,
3,782611,canvasDisplay,"{'canvasName': 'MainCanvas', 'canvasChildObjec...",SimpleMaze,1,0,InitTrial,
4,782611,canvasDisplay,"{'canvasName': 'MainCanvas', 'canvasChildObjec...",SimpleMaze,1,0,InitTrial,



------------------
events.event_times


,subj_sess,trial,trial_phase,gameState,index,time,duration,time_bins,time_bin_dur,n_time_bins
0,U518_ses0,1,1,Delay1,"[6, 22]","[782611, 792694]",10083,"[782611, 783111, 783611, 784111, 784611, 78511...",10000,20
1,U518_ses0,1,2,Encoding,"[22, 922]","[792694, 822709]",30015,"[792694, 793194, 793694, 794194, 794694, 79519...",30000,60
2,U518_ses0,1,3,ReturnToBase1,"[922, 999]","[822709, 825127]",2418,"[822709, 823209, 823709, 824209, 824709]",2000,4
3,U518_ses0,1,4,Delay2,"[999, 1015]","[825127, 835143]",10016,"[825127, 825627, 826127, 826627, 827127, 82762...",10000,20
4,U518_ses0,1,5,Retrieval,"[1015, 1904]","[835143, 865159]",30016,"[835143, 835643, 836143, 836643, 837143, 83764...",30000,60



----------------
events.positions


...

,time,trial,time_penalty,gameState,bad_trials,gold_id,gold_x,gold_z
0,792694,1,0,Encoding,,gold0001,26.393890,9.762549
1,879958,2,1,Encoding,,gold0002,12.096540,12.546080
2,962657,3,1,Encoding,,gold0003,21.740080,21.299550
3,1042722,4,1,Encoding,,gold0004,17.204920,11.904680
4,1130403,5,1,Encoding,,gold0005,6.764997,26.321900



----------------
events.dig_events


,time,trial,time_penalty,gameState,bad_trials,gold_id,dig_success,dist_from_gold
0,1009773,3,1,Retrieval,,gold0003,True,0.915762
1,1099337,4,1,Retrieval,,gold0004,False,12.552280
2,1103204,4,1,Retrieval,,gold0004,False,14.139950
3,1200553,5,1,Retrieval,,gold0005,True,0.730915
4,1283984,6,1,Retrieval,,gold0006,False,6.999308



----------------
events.events_behav


,subj,sess,trial,time_penalty,gameState,start_time,stop_time,time_bin,time_step,maze_region,head_direc,is_moving,base_in_view,gold_in_view,gold_in_view_ids,dig_performed,dig_success,all_digs,dig_ids,dig_dists
0,U518,ses0,1,0,Delay1,782611,783111,0,1,Base,N,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U518,ses0,1,0,Delay1,783111,783611,1,1,Base,N,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U518,ses0,1,0,Delay1,783611,784111,2,2,Base,N,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,U518,ses0,1,0,Delay1,784111,784611,3,2,Base,N,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U518,ses0,1,0,Delay1,784611,785111,4,3,Base,N,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# events also contains a dataframe that gives the x, y coordinates
# and region labels for the virtual environment that subjects navigated
# during the main (post-tutorial) portion of the experiment.
print(events.maze, end='\n'*2)

display(events.maze.maze.head())

SimpleMaze
----------
27 x 29 m
83 tiles (783 m^2)
1 floor tiles in the base (35 m^2)
66 floor tiles in the mine (540 m^2)
12 inner wall tiles (100 m^2)
4 outer wall tiles (108 m^2)



,coords,hem,region,proxim,idx,region_precise,area,dist_from_spawn
0,"[(4, 1), (7, 4)]",L,hall,near,0,SW Hall,9,11.036757
1,"[(7, 1), (10, 4)]",L,hall,near,1,SW Hall,9,8.050466
2,"[(10, 1), (13, 4)]",L,hall,near,2,SW Hall,9,5.080354
3,"[(20, 1), (23, 4)]",R,hall,near,2,SE Hall,9,5.080354
4,"[(23, 1), (26, 4)]",R,hall,near,1,SE Hall,9,8.050466


In [11]:
# event_spikes combines the events.events_behav dataframe with spike counts
# for all units in the session.
event_spikes = time_bin_analysis.load_event_spikes(subj_sess, proj_dir=proj_dir)

print(event_spikes)

# 4-1 is the first unit from channel 4, 4-2 is the second unit from channel 4, etc.
print('neurons:', event_spikes.column_map['neurons'], end='\n'*2)

display(event_spikes.event_spikes.head())

Loading saved EventSpikes file
36 neurons

neurons: ['4-1', '4-2', '9-1', '11-1', '13-1', '13-2', '15-1', '16-1', '16-2', '25-1', '27-1', '28-1', '30-1', '31-1', '32-1', '34-1', '35-1', '36-1', '37-1', '38-1', '38-2', '39-1', '40-1', '42-1', '44-1', '65-1', '67-1', '67-2', '68-1', '70-1', '72-1', '73-1', '78-1', '78-2', '78-3', '79-1']



,subj,sess,trial,time_penalty,gameState,start_time,stop_time,time_bin,is_moving,time_step,maze_region,head_direc,base_in_view,gold_in_view,gold_in_view_ids,dig_performed,dig_success,all_digs,dig_ids,dig_dists,icpt,time-1,time-2,time-3,time-4,time-5,time-6,time-7,time-8,time-9,time-10,time-11,time-12,time-13,time-14,time-15,time-16,time-17,time-18,time-19,time-20,time-21,time-22,time-23,time-24,time-25,time-26,time-27,time-28,time-29,time-30,time-31,place-Base,place-C_Hall,place-NE_Hall,place-NE_Room,place-NW_Hall,place-NW_Room,place-N_Passage,place-SE_Hall,place-SE_Room,place-SW_Hall,place-SW_Room,place-S_Passage,hd-E,hd-N,hd-NE,hd-NW,hd-S,hd-SE,hd-SW,hd-W,trial-1,trial-2,trial-3,trial-4,trial-5,trial-6,trial-7,trial-8,trial-9,trial-10,trial-11,trial-12,trial-13,trial-14,trial-15,trial-16,trial-17,trial-18,trial-19,trial-20,trial-21,trial-22,trial-23,trial-24,trial-25,trial-26,trial-27,trial-28,trial-29,trial-30,trial-31,trial-32,trial-33,trial-34,trial-35,trial-36,4-1,4-2,9-1,11-1,13-1,13-2,15-1,16-1,16-2,25-1,27-1,28-1,30-1,31-1,32-1,34-1,35-1,36-1,37-1,38-1,38-2,39-1,40-1,42-1,44-1,65-1,67-1,67-2,68-1,70-1,72-1,73-1,78-1,78-2,78-3,79-1
0,U518,ses0,1,0,Delay1,782611,783111,0,0,1,Base,N,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,2,7,1,8,3,0,1,1,0,0,0,3,0,0,4,0,0,1,3,2,9,2,7,7,2,2,2,3,1,4,0,1,0
1,U518,ses0,1,0,Delay1,783111,783611,1,0,1,Base,N,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2,1,5,8,2,1,1,0,1,2,0,1,1,2,0,3,6,0,1,0,0,6,5,1,1,7,2,4,1,1,4,5,0,0,4
2,U518,ses0,1,0,Delay1,783611,784111,2,0,2,Base,N,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,1,3,1,4,0,4,6,0,0,2,0,0,1,1,0,5,7,0,1,0,1,5,3,1,0,2,0,0,1,1,1,1,1,1,1
3,U518,ses0,1,0,Delay1,784111,784611,3,0,2,Base,N,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,3,3,1,6,1,0,0,3,0,0,0,1,0,2,8,0,0,2,3,2,3,0,2,3,0,6,1,2,2,0,0,0,3
4,U518,ses0,1,0,Delay1,784611,785111,4,0,3,Base,N,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,12,3,3,1,0,1,2,2,2,1,2,0,2,6,2,0,2,3,2,10,0,1,3,1,4,0,2,4,1,0,0,3


In [14]:
# Here's how you can load just the spike data for some unit, in case that's the format you want it.
neuron = '4-1'
spikes = spike_preproc.load_spikes(subj_sess, neuron, proj_dir=proj_dir)
spikes

subj_sess                                                 U518_ses0
subj                                                           U518
sess                                                           ses0
chan                                                              4
unit                                                              1
sess_stop_time                                              4866550
spike_times       [62, 1105, 1344, 1510, 1535, 1538, 1567, 1695,...
n_spikes                                                      24446
fr                                                          5.02327
hemroi                                                          REC
Name: 0, dtype: object

In [15]:
# pop_spikes combines part of event_spikes for all sessions in the study,
# and is used for training classifiers to predict time from cross-subject neural
# firing data.
pop_spikes = pop_decoding.load_pop_spikes(proj_dir=proj_dir)

print(pop_spikes, end='\n'*2)

display(pop_spikes.pop_spikes)

457 neurons
500 ms time bins
36 trials
160 time bins/trial
pop_spikes: (5760, 460)



,trial,gameState,time_bin,U518_ses0-4-1,U518_ses0-4-2,U518_ses0-9-1,U518_ses0-11-1,U518_ses0-13-1,U518_ses0-13-2,U518_ses0-15-1,U518_ses0-16-1,U518_ses0-16-2,U518_ses0-25-1,U518_ses0-27-1,U518_ses0-28-1,U518_ses0-30-1,U518_ses0-31-1,U518_ses0-32-1,U518_ses0-34-1,U518_ses0-35-1,U518_ses0-36-1,U518_ses0-37-1,U518_ses0-38-1,U518_ses0-38-2,U518_ses0-39-1,U518_ses0-40-1,U518_ses0-42-1,U518_ses0-44-1,U518_ses0-73-1,U518_ses0-78-1,U518_ses0-78-2,U518_ses0-78-3,U518_ses0-79-1,U518_ses1-4-1,U518_ses1-4-3,U518_ses1-4-4,U518_ses1-9-1,U518_ses1-11-1,U518_ses1-12-1,U518_ses1-13-1,U518_ses1-13-2,U518_ses1-14-1,U518_ses1-15-1,U518_ses1-15-2,U518_ses1-34-1,U518_ses1-35-1,U518_ses1-35-2,U518_ses1-36-1,U518_ses1-36-2,U518_ses1-37-1,U518_ses1-37-2,U518_ses1-38-1,U518_ses1-39-1,U518_ses1-40-1,U518_ses1-40-2,U518_ses1-41-1,U518_ses1-41-2,U518_ses1-42-1,U518_ses1-43-1,U518_ses1-43-2,U518_ses1-43-3,U518_ses1-73-1,U518_ses1-73-2,U518_ses1-73-3,U518_ses1-74-1,U518_ses1-76-1,U518_ses1-77-1,U518_ses1-78-1,U518_ses1-79-1,U521_ses0-5-1,U521_ses0-17-1,U521_ses0-17-2,U521_ses0-20-1,U521_ses0-21-2,U521_ses0-22-1,U521_ses0-23-1,U521_ses0-24-1,U521_ses0-69-1,U521_ses0-71-1,U521_ses0-18-1,U521_ses0-18-2,U521_ses0-18-3,U521_ses0-19-1,U521_ses0-19-2,U521_ses0-19-3,U521_ses0-19-4,U521_ses0-21-1,U521_ses0-21-3,U521_ses0-22-2,U521_ses0-23-2,U521_ses0-24-2,U521_ses0-70-1,U521_ses0-71-2,U526_ses0-12-1,U526_ses0-14-1,U526_ses0-15-1,U526_ses0-17-2,U526_ses0-21-1,U526_ses0-23-2,U526_ses0-27-1,U526_ses0-41-1,U526_ses0-41-3,U526_ses0-42-2,U526_ses0-43-2,U526_ses0-45-1,U526_ses0-47-2,U526_ses0-12-2,U526_ses0-14-2,U526_ses0-17-1,U526_ses0-17-3,U526_ses0-23-1,U526_ses0-24-1,U526_ses0-38-1,U526_ses0-41-2,U526_ses0-42-1,U526_ses0-43-1,U526_ses0-44-1,U526_ses0-47-1,U527_ses0-9-2,U527_ses0-11-2,U527_ses0-13-1,U527_ses0-16-1,U527_ses0-17-1,U527_ses0-18-1,U527_ses0-20-2,U527_ses0-41-1,U527_ses0-43-2,U527_ses0-44-2,U527_ses0-44-4,U527_ses0-45-2,U527_ses0-57-1,U527_ses0-58-2,U527_ses0-60-1,U527_ses0-62-1,U527_ses0-9-1,U527_ses0-11-1,U527_ses0-12-1,U527_ses0-13-2,U527_ses0-16-2,U527_ses0-17-2,U527_ses0-20-1,U527_ses0-23-1,U527_ses0-43-1,U527_ses0-44-1,U527_ses0-44-3,U527_ses0-45-1,U527_ses0-48-1,U527_ses0-58-1,U527_ses0-59-1,U527_ses0-61-1,U527_ses0-63-1,U530_ses0-2-1,U530_ses0-2-2,U530_ses0-3-1,U530_ses0-3-2,U530_ses0-4-1,U530_ses0-5-1,U530_ses0-5-2,U530_ses0-7-1,U530_ses0-7-2,U530_ses0-7-3,U530_ses0-7-4,U530_ses0-8-1,U530_ses0-8-2,U530_ses0-8-3,U530_ses0-9-1,U530_ses0-13-1,U530_ses0-13-2,U530_ses0-15-1,U530_ses0-20-1,U530_ses0-20-2,U530_ses0-22-1,U530_ses0-26-1,U530_ses0-33-1,U530_ses0-41-1,U530_ses0-41-2,U530_ses0-41-3,U530_ses0-42-1,U530_ses0-43-1,U530_ses0-43-2,U530_ses0-44-1,U530_ses0-44-2,U530_ses0-45-1,U530_ses0-45-2,U530_ses0-46-1,U530_ses0-46-2,U530_ses0-46-3,U530_ses0-47-1,U530_ses0-47-2,U530_ses0-48-1,U530_ses0-48-2,U530_ses0-48-3,U530_ses0-58-1,U530_ses0-60-1,U530_ses0-65-1,U530_ses0-66-1,U530_ses0-66-2,U530_ses0-67-1,U530_ses0-67-2,U530_ses0-68-1,U530_ses0-69-1,U530_ses0-69-2,U530_ses0-70-1,U530_ses0-70-2,U530_ses0-71-1,U530_ses0-72-1,U530_ses0-72-2,U530_ses0-73-1,U530_ses0-73-2,U530_ses0-74-1,U530_ses0-75-1,U530_ses0-76-1,U530_ses0-76-2,U530_ses0-77-1,U530_ses0-80-1,U530_ses0-80-2,U530_ses0-82-1,U530_ses0-82-2,U530_ses1-9-1,U530_ses1-13-1,U530_ses1-15-1,U530_ses1-16-1,U530_ses1-16-2,U530_ses1-16-3,U530_ses1-41-1,U530_ses1-41-2,U530_ses1-41-3,U530_ses1-42-1,U530_ses1-42-2,U530_ses1-44-1,U530_ses1-46-1,U530_ses1-46-2,U530_ses1-47-1,U530_ses1-47-2,U530_ses1-58-1,U530_ses1-67-1,U530_ses1-69-1,U530_ses1-69-2,U530_ses1-71-1,U530_ses1-71-2,U530_ses1-71-3,U530_ses1-73-1,U530_ses1-73-2,U530_ses1-76-1,U530_ses1-76-2,U530_ses1-76-3,U530_ses1-76-4,U530_ses1-77-1,U530_ses1-79-1,U530_ses1-79-2,U530_ses1-80-1,U530_ses1-80-2,U530_ses1-80-3,U532_ses0-2-1,U532_ses0-2-2,U532_ses0-2-3,U532_ses0-3-1,U532_ses0-3-2,U532_ses0-3-3,U532_ses0-4-1,U532_ses0-4-2,U532_ses0-4-3,U532_ses0-5-1,U532_ses0-5-2,U532_ses0-5-3,U532_ses0-5-4,U532_ses0-5-5,U532_ses0-5-6,U532_ses0-5-7,U532_ses0-5-8,U532_ses0-6-